# LangChain + xpander.ai Integration Example

This notebook demonstrates how to integrate **LangChain** with **xpander.ai** to create powerful AI agents with access to pre-built tools and services.

🚀 **Quick Start Template**: [Import this template](https://app.xpander.ai/templates/13de363d-c947-41d9-a25a-3e94d2fad90f) directly into xpander.ai

## Prerequisites

- **xpander.ai account required**: [Sign up here](https://app.xpander.ai)
- OpenAI API key
- xpander.ai API credentials

## What You'll Learn

- How to set up LangChain with xpander.ai tools
- How to create a ReAct agent using LangGraph
- How to leverage xpander's pre-built tool ecosystem


## 1. Installation & Setup

Install the required dependencies:

In [ ]:
# Install dependencies
!pip install "xpander-sdk" "langchain[openai]" langgraph python-dotenv

## 2. Environment Configuration

Load environment variables from `.env` file:

In [ ]:
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

Validate that all required environment variables are set:

In [ ]:
from os import getenv

# Required environment variables
required_env_vars = [
    "OPENAI_API_KEY",      # Your OpenAI API key
    "XPANDER_API_KEY",     # Your xpander.ai API key
    "XPANDER_ORGANIZATION_ID",  # Your xpander.ai organization ID
    "XPANDER_AGENT_ID"     # Your xpander.ai agent ID
]

# Validate environment variables
for evar in required_env_vars:
    if not getenv(evar, None):
        raise KeyError(f"Missing required environment variable: {evar}")
        
print("✅ All required environment variables are set!")

## 3. Initialize xpander.ai Agent

Load the xpander agent configuration with pre-built tools:

In [ ]:
from os import getenv
from xpander_sdk import Agents

# Initialize xpander agent
xpander_agent = Agents().get(agent_id=getenv("XPANDER_AGENT_ID"))

print(f"📦 Loaded xpander agent with {len(xpander_agent.tools.functions)} tools")
print(f"🤖 Using model: {xpander_agent.model_name}")

## 4. Create LangChain ReAct Agent

Set up a LangChain agent using LangGraph with xpander.ai tools:

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage

# Initialize the language model
llm = ChatOpenAI(model=xpander_agent.model_name, temperature=0)

# Extract system prompt from xpander agent instructions
def create_system_prompt(instructions):
    """Convert xpander agent instructions to system prompt"""
    parts = []
    
    if hasattr(instructions, 'general') and instructions.general:
        parts.append(f"System: {instructions.general}")
    
    if hasattr(instructions, 'goal_str') and instructions.goal_str:
        parts.append(f"Goals:\n{instructions.goal_str}")
    
    if hasattr(instructions, 'instructions') and instructions.instructions:
        instr_list = "\n".join([f"- {instr}" for instr in instructions.instructions])
        parts.append(f"Instructions:\n{instr_list}")
    
    return "\n\n".join(parts)

# Create system prompt from xpander agent
system_prompt = create_system_prompt(xpander_agent.instructions)
print(f"📋 System Prompt:\n{system_prompt}\n")

# Create a ReAct agent with xpander tools and system prompt
agent = create_react_agent(
    llm, 
    xpander_agent.tools.functions
)

print(f"🚀 LangChain ReAct agent created with {len(xpander_agent.tools.functions)} xpander.ai tools and system instructions!")

## 5. Demo: Agent in Action

Let's test the agent with a sample query to demonstrate the tool usage:

In [ ]:
# Demo: Test the agent with a sample query
prompt = "search for information about xpander.ai"

print(f"📋 User Query: {prompt}")
print("\n🚀 Agent Processing...\n")

# Stream the agent's response
for chunk in agent.stream({"messages": [("system", system_prompt),("user", prompt)]}):
    # Extract the final message content
    if "agent" in chunk and chunk["agent"].get("messages"):
        messages = chunk["agent"]["messages"]
        for message in messages:
            if hasattr(message, 'content') and message.content:
                print(f"🤖 Agent: {message.content}")


## 6. Advanced Features

This example demonstrates a LangChain agent with access to **xpander.ai tools**:

- **🔍 Tavily Search**: Advanced web search with configurable output schema filtering
- **📧 Send Email**: Email sending capabilities

### Tool Output Filtering

xpander.ai provides advanced [output schema filtering](https://docs.xpander.ai/user-guide/backend-configuration/connectors#tool-scheme-advanced-tab) for tools, allowing you to:

- Reduce token usage by filtering unnecessary data
- Remove personally identifiable information (PII)
- Focus the AI model on relevant response parts
- Prevent exposure of sensitive data

### Tool Dependencies

You can configure tool dependencies in the xpander.ai platform to ensure proper execution order.

## 7. Next Steps

Ready to build your own LangChain + xpander.ai integration? Here's what to do next:

1. **🚀 Import Template**: [Use this template](https://app.xpander.ai/templates/13de363d-c947-41d9-a25a-3e94d2fad90f) as a starting point
2. **🔧 Customize Tools**: Add more tools from xpander.ai's library in your agent configuration
3. **⚙️ Configure Filtering**: Set up output schema filtering for your tools
4. **📜 Read the Docs**: Learn more at [docs.xpander.ai](https://docs.xpander.ai)
5. **💬 Join Community**: Connect with other builders on [Discord](https://discord.gg/CUcp4WWh5g)

---

**Happy building! 🎆**